In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score, make_scorer
import pickle


In [ ]:
df = pd.read_csv("../data/nfip_claims_ML.csv")

In [ ]:
df.columns

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df.drop(["amountPaidonTotalClaim", "yearOfLoss", "originalConstructionDate", "Unnamed: 0", "floodZone", "LossRatio", "amountPaidOnBuildingClaim", "amountPaidOnContentsClaim", "amountPaidOnIncreasedCostOfComplianceClaim", "reportedZipcode", "latitude", "longitude", "countyCode"], inplace=True, axis=1)
df.claim_segmentation = [0 if x==1 else 1 for x in df.claim_segmentation]

In [ ]:
#df = pd.get_dummies(df, drop_first=True)
#df.head()

In [ ]:
df.to_csv("../data/claims_dummied.csv", index=False)

In [ ]:
df.head()

In [ ]:
df.claim_segmentation

### In the next cell, make sure to remove the target column from the appropiate list! 

In [ ]:
# create lists of features that need encoding or scaling
cat_features = list(df.select_dtypes(exclude="number").columns)
num_features = list(df.select_dtypes(include="number").columns)
num_features.remove("claim_segmentation")

In [ ]:
# create a column transformer out of a scaler for the numerical and an encoder for the categorical columns 
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer([
    ("num", scaler, num_features),
    ("cat", encoder, cat_features)
])

In [ ]:
# build pipe and name its components
pipe = Pipeline([
("preprocessor", preprocessor),
("clf", RandomForestClassifier(random_state=42))
])

### Remove cell/adjust target name as appropiate! 

In [ ]:
# train test split
y = df.pop("claim_segmentation")
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# make fbeta scorer
#ftwo_scorer = make_scorer(fbeta_score, beta=2)

In [ ]:
# define params for random search
rs_params={"clf__max_depth": list(np.arange(10, 100, step=10)) + [None],
              "clf__n_estimators": np.arange(50, 250, step=50),
              "clf__max_features": ["sqrt", "log2"],
              "clf__criterion": ["gini","entropy"],
              "clf__min_samples_leaf": np.arange(2, 10),
              "clf__min_samples_split": np.arange(2, 10, step=2)
          }

In [ ]:
# define randomized search
rand = RandomizedSearchCV(pipe, rs_params, n_iter=1, scoring="accuracy", cv=5, n_jobs=-1, random_state=42, verbose=5)

In [ ]:
# run randomized search
rand.fit(X_train, y_train)

In [ ]:
rand

In [ ]:
# show best parameters
print("Best score:\n{:.2f}".format(rand.best_score_))
print("Best parameters:\n{}".format(rand.best_params_))

In [ ]:
# create a results dataframe from the results dict of the classifier
result_df_rand = pd.DataFrame.from_dict(rand.cv_results_, orient="columns")

# plot some results, e.g.
sns.relplot(data=result_df_rand,
            kind="line",
            x="param_clf__n_estimators",
            y="mean_test_score",
            hue="param_clf__max_features",
            col="clf__criterion")
plt.show()

In [ ]:
result_df_rand

In [ ]:
pd.to_csv("../data/random_search_results.csv")

In [ ]:
# define params for grid search, based on random search results
gs_params = {
               }

In [ ]:
# define grid search
grid = GridSearchCV(pipe, param_grid=gs_params, cv=5, scoring=ftwo_scorer, verbose=1, n_jobs=-1)

In [ ]:
# run grid search
grid.fit(X_train, y_train)

In [ ]:
# show best parameters
print("Best score:\n{:.2f}".format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))

In [ ]:
# save best model
best_model = grid.best_estimator_
filename = "finalized_model.sav"
pickle.dump(best_model, open(filename, "wb"))